<a href="https://colab.research.google.com/github/kim-jason/PowerliftingDataset/blob/main/Powerlifting_Data_Science_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Powerlifting Dataset EDA

Jason Kim  <br>
Email: jason.s.kim20@gmail.com <br>
Linkedin: https://www.linkedin.com/in/jasonks/

Introduction to Powerlifting <br>

TO DO: Explain history of powerlifting, the three lifts


Goals of this notebook:

1.   See progression over time
2.   Difference between countries



Data import

In [1]:
import pandas as pd

In [3]:
# Download the raw CSV dataset
! gdown --id 1b15_F2lH2sHI02m7F1HGQkDw3Bl25-X9

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1b15_F2lH2sHI02m7F1HGQkDw3Bl25-X9
To: /content/openpowerlifting-2022-08-06-98216f16.csv
100% 540M/540M [00:02<00:00, 228MB/s]


In [5]:
df = pd.read_csv('openpowerlifting-2022-08-06-98216f16.csv')
df.head()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (33,35,38) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Name,Sex,Event,Equipment,Age,AgeClass,BirthYearClass,Division,BodyweightKg,WeightClassKg,...,Tested,Country,State,Federation,ParentFederation,Date,MeetCountry,MeetState,MeetTown,MeetName
0,Alona Vladi,F,SBD,Raw,33.0,24-34,24-39,O,58.30,60,...,Yes,Russia,NaN,GFP,NaN,2019-05-11,Russia,NaN,Bryansk,Open Tournament
1,Galina Solovyanova,F,SBD,Raw,43.0,40-44,40-49,M1,73.10,75,...,Yes,Russia,NaN,GFP,NaN,2019-05-11,Russia,NaN,Bryansk,Open Tournament
2,Daniil Voronin,M,SBD,Raw,15.5,16-17,14-18,T,67.40,75,...,Yes,Russia,NaN,GFP,NaN,2019-05-11,Russia,NaN,Bryansk,Open Tournament
3,Aleksey Krasov,M,SBD,Raw,35.0,35-39,24-39,O,66.65,75,...,Yes,Russia,NaN,GFP,NaN,2019-05-11,Russia,NaN,Bryansk,Open Tournament
4,Margarita Pleschenkova,M,SBD,Raw,26.5,24-34,24-39,O,72.45,75,...,Yes,Russia,NaN,GFP,NaN,2019-05-11,Russia,NaN,Bryansk,Open Tournament


In [9]:
df.describe().apply(lambda s: s.apply('{0:.1f}'.format))


,Age,BodyweightKg,Squat1Kg,Squat2Kg,Squat3Kg,Squat4Kg,Best3SquatKg,Bench1Kg,Bench2Kg,Bench3Kg,...,Deadlift1Kg,Deadlift2Kg,Deadlift3Kg,Deadlift4Kg,Best3DeadliftKg,TotalKg,Dots,Wilks,Glossbrenner,Goodlift
count,1720150.0,2719385.0,710550.0,702587.0,682372.0,7313.0,1821444.0,1158272.0,1144372.0,1107394.0,...,810608.0,795214.0,759616.0,19399.0,1970884.0,2564651.0,2541034.0,2541034.0,2541034.0,2318067.0
mean,30.8,84.1,113.5,95.8,36.6,75.7,174.3,83.5,56.3,-16.5,...,159.6,131.8,20.7,80.1,189.1,375.4,273.0,272.0,256.5,63.9
std,13.2,22.7,140.2,164.7,193.0,182.8,69.0,101.3,126.2,141.0,...,108.3,157.4,211.4,187.3,62.2,206.0,129.6,129.1,122.7,16.3
min,0.0,10.0,-555.0,-580.0,-600.5,-550.0,-508.0,-515.0,-589.7,-575.0,...,-461.0,-502.5,-587.5,-500.0,-410.0,1.0,0.7,0.7,0.6,0.5
25%,20.5,67.1,90.0,75.0,-155.0,-88.0,122.5,55.0,-45.0,-135.0,...,125.0,115.0,-202.5,-75.5,140.0,200.0,141.6,141.0,133.5,52.2
50%,27.5,81.9,145.0,145.0,115.0,131.0,170.0,105.0,95.0,-60.0,...,180.0,177.5,120.0,142.9,188.2,351.5,294.6,293.6,274.1,63.6
75%,38.5,98.8,200.0,202.5,192.5,198.5,217.7,145.0,145.0,117.5,...,224.5,230.0,205.0,206.4,235.0,530.7,371.1,369.5,350.0,75.2
max,98.0,300.0,555.0,577.5,595.0,592.4,595.0,551.0,500.0,577.5,...,450.0,467.5,487.5,440.5,487.5,1407.5,795.2,793.3,756.9,167.5


In [10]:
df.dtypes

Name                 object
Sex                  object
Event                object
Equipment            object
Age                 float64
AgeClass             object
BirthYearClass       object
Division             object
BodyweightKg        float64
WeightClassKg        object
Squat1Kg            float64
Squat2Kg            float64
Squat3Kg            float64
Squat4Kg            float64
Best3SquatKg        float64
Bench1Kg            float64
Bench2Kg            float64
Bench3Kg            float64
Bench4Kg            float64
Best3BenchKg        float64
Deadlift1Kg         float64
Deadlift2Kg         float64
Deadlift3Kg         float64
Deadlift4Kg         float64
Best3DeadliftKg     float64
TotalKg             float64
Place                object
Dots                float64
Wilks               float64
Glossbrenner        float64
Goodlift            float64
Tested               object
Country              object
State                object
Federation           object
ParentFederation    

TO DO: Explain each feature

adlskfjafksdj

### Data cleaning